In [2]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import matplotlib.pyplot as plt
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import numpy as np
np.random.seed(0)
import itertools
from keras.preprocessing.image import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

<h1>Data Loading</h1>
<p>Setting up Image Data Generators<p>

In [3]:
#train_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/train",image_size=(256, 256))
#test_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/valid",image_size=(256, 256))

train_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                         image_size=(256, 256))
test_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
                                        image_size=(256, 256))

rescale = Rescaling(scale=1.0/255)
train_gen = train_gen.map(lambda image,label:(rescale(image),label))
test_gen  = test_gen.map(lambda image,label:(rescale(image),label))

Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.


<h1>Modelling</h1>

In [13]:
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Load pretrained ResNet50 as the base (exclude the fully connected top layers)
base_model = ResNet50(
    weights='imagenet',        # pretrained on ImageNet
    include_top=False,         # drop the original FC layers
    input_shape=(256, 256, 3)  # your input shape
)

# Freeze the base model initially
base_model.trainable = False

# Build the model
model = models.Sequential([
    base_model,                          # feature extractor
    layers.GlobalAveragePooling2D(),     # better than Flatten for ResNet
    layers.Dense(512, activation='relu'),
    layers.Dense(38, activation='softmax')  # 38 classes
])

# Compile
opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dense_5 (Dense)              (None, 38)                19494     
Total params: 24,656,294
Trainable params: 1,068,582
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
ep = 10
history = model.fit_generator(train_gen,
          validation_data=test_gen,
          epochs = ep)

Epoch 1/10
2197/2197 [==============================] - 201s 90ms/step - loss: 3.3563 - accuracy: 0.1131 - val_loss: 3.0988 - val_accuracy: 0.2024
Epoch 2/10
2197/2197 [==============================] - 198s 90ms/step - loss: 2.9418 - accuracy: 0.2246 - val_loss: 2.7849 - val_accuracy: 0.2634
Epoch 3/10
 377/2197 [====>.........................] - ETA: 2:11 - loss: 2.7681 - accuracy: 0.2620

<h1>Metrics</h1>

In [9]:
labels = []
predictions = []
for x,y in test_gen:
    labels.append(list(y.numpy()))
    predictions.append(tf.argmax(model.predict(x),1).numpy())

In [10]:
predictions = list(itertools.chain.from_iterable(predictions))
labels = list(itertools.chain.from_iterable(labels))

In [11]:
print("Train Accuracy  : {:.2f} %".format(history.history['accuracy'][-1]*100))
print("Test Accuracy   : {:.2f} %".format(accuracy_score(labels, predictions) * 100))
print("Precision Score : {:.2f} %".format(precision_score(labels, predictions, average='micro') * 100))
print("Recall Score    : {:.2f} %".format(recall_score(labels, predictions, average='micro') * 100))

Train Accuracy  : 34.64 %
Test Accuracy   : 40.15 %
Precision Score : 40.15 %
Recall Score    : 40.15 %
